In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf

# Making multiple image tiles from single tile

In [ ]:
!pip install geotile==0.2.3

In [ ]:
from geotile import GeoTile

Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


Creating tiles in several folders at once

In [ ]:
import os

Five tiles are kept in five folders and five mask files are kept in five folders.

In [ ]:
folder_paths = [
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Tiles/One',          #Respective folder(s) in the drive
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Tiles/Two',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Tiles/Three',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Tiles/Four',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Tiles/Five',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Masks/One',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Masks/Three',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Masks/Two',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Masks/Four',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Masks/Five'
    # Add more folder paths as needed
]
# Iterate over each folder path and generate the tiles
for folder_path in folder_paths:
    os.chdir(folder_path)
    image_files = os.listdir(folder_path)
    # Create the output folder corresponding to the input folder
    output_folder = os.path.join(folder_path, 'output')
    os.makedirs(output_folder, exist_ok=True)

    for image_file in image_files:
        if image_file.endswith('.tif'):  # Modify the extension if needed
            gt = GeoTile(image_file)
            gt.generate_tiles(output_folder)


Tiles from folder number one, two, four, and five are used for training and therefore are stored those in a sigle folder named 'train'. Tiles from the folder number three are used for testing and therefore are kept in the folder named 'test'. Both in train and test folders, there are two folders. One (images)is for storing the images and another (masks) is for storing the masks.

Checking the information of the image

In [ ]:
os.chdir(r"/content/gdrive/MyDrive/Colab Notebooks/Data_Augmentation/train/images")         #Respective folder(s) in the drive
gt = GeoTile('75.tif')

In [ ]:
gt.height

256

In [ ]:
gt.width

256

In [ ]:
gt.meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': 0.0,
 'width': 256,
 'height': 256,
 'count': 3,
 'crs': CRS.from_epsg(32611),
 'transform': Affine(0.4994727339500307, 0.0, 464111.1503432652,
        0.0, -0.4993928845171347, 5919139.720065584)}

#Data Augmentation

Import necessary libraries

In [ ]:
import cv2
from tqdm import tqdm
from glob import glob
from albumentations import CenterCrop, RandomRotate90, GridDistortion, HorizontalFlip, VerticalFlip

Function for loading input data

In [ ]:
def load_data(path):                                                      #Respective folder(s) in the drive
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))
    return images, masks

Function for creating output directory

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
import tifffile

Data augmentation for the training set

In [ ]:
import tifffile

def augment_data(images, masks, save_path, augment=True):
    H = 256
    W = 256

    for x, y in tqdm(zip(images, masks), total=len(images)):
        name = x.split("/")[-1].split(".")
        """ Extracting the name and extension of the image and the mask. """
        image_name = name[0]
        image_extn = name[1]

        name = y.split("/")[-1].split(".")
        mask_name = name[0]
        mask_extn = name[1]

        """ Reading image and mask. """
        x = tifffile.imread(x)
        y = tifffile.imread(y)

        """ Augmentation """
        if augment == True:
            aug = CenterCrop(H, W, p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            aug = GridDistortion(p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']

            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x4 = augmented['image']
            y4 = augmented['mask']

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x5 = augmented['image']
            y5 = augmented['mask']

            save_images = [x, x1, x2, x3, x4, x5]
            save_masks =  [y, y1, y2, y3, y4, y5]

        else:
            save_images = [x]
            save_masks = [y]

        """ Saving the image and mask. """
        idx = 0
        for i, m in zip(save_images, save_masks):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            if len(images) == 1:
                tmp_img_name = f"{image_name}_i.{image_extn}"
                tmp_mask_name = f"{mask_name}_l.{mask_extn}"
            else:
                tmp_img_name = f"{image_name}_{idx}_i.{image_extn}"
                tmp_mask_name = f"{mask_name}_{idx}_l.{mask_extn}"

            image_path = os.path.join(save_path, "images", tmp_img_name)               
            mask_path = os.path.join(save_path, "masks", tmp_mask_name)                

            tifffile.imwrite(image_path, i)
            tifffile.imwrite(mask_path, m)

            idx += 1

if __name__ == "__main__":
    """ Loading original images and masks. """
    path = "/content/gdrive/MyDrive/Colab Notebooks/Data_Augmentation/train"                      #Respective folder(s) in the drive
    images, masks = load_data(path)
    print(f"Original Images: {len(images)} - Original Masks: {len(masks)}")

    """ Creating folders. """
    create_dir("/content/gdrive/MyDrive/Colab Notebooks/new_data_train/images")
    create_dir("/content/gdrive/MyDrive/Colab Notebooks/new_data_train/masks")

    """ Applying data augmentation. """
    augment_data(images, masks, "/content/gdrive/MyDrive/Colab Notebooks/new_data_train", augment=True)

    """ Loading augmented images and masks. """
    images, masks = load_data("/content/gdrive/MyDrive/Colab Notebooks/new_data_train/")
    print(f"Augmented Images: {len(images)} - Augmented Masks: {len(masks)}")

Original Images: 100 - Original Masks: 100


 39%|███▉      | 39/100 [00:18<00:27,  2.21it/s]Exception ignored in: <function GeoTile.__del__ at 0x7fc7a54b0940>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/geotile/GeoTile.py", line 66, in __del__
    self.ds.close()
AttributeError: 'GeoTile' object has no attribute 'ds'
100%|██████████| 100/100 [00:48<00:00,  2.06it/s]

Augmented Images: 600 - Augmented Masks: 600


Data augmentation for testing set

In [ ]:
import tifffile

def augment_data(images, masks, save_path, augment=True):
    H = 256
    W = 256

    for x, y in tqdm(zip(images, masks), total=len(images)):
        name = x.split("/")[-1].split(".")
        """ Extracting the name and extension of the image and the mask. """
        image_name = name[0]
        image_extn = name[1]

        name = y.split("/")[-1].split(".")
        mask_name = name[0]
        mask_extn = name[1]

        """ Reading image and mask. """
        x = tifffile.imread(x)
        y = tifffile.imread(y)

        """ Augmentation """
        if augment == True:
            aug = CenterCrop(H, W, p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            aug = GridDistortion(p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']

            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x4 = augmented['image']
            y4 = augmented['mask']

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x5 = augmented['image']
            y5 = augmented['mask']

            save_images = [x, x1, x2, x3, x4, x5]
            save_masks =  [y, y1, y2, y3, y4, y5]

        else:
            save_images = [x]
            save_masks = [y]

        """ Saving the image and mask. """
        idx = 0
        for i, m in zip(save_images, save_masks):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            if len(images) == 1:
                tmp_img_name = f"{image_name}_i.{image_extn}"
                tmp_mask_name = f"{mask_name}_l.{mask_extn}"
            else:
                tmp_img_name = f"{image_name}_{idx}_i.{image_extn}"
                tmp_mask_name = f"{mask_name}_{idx}_l.{mask_extn}"

            image_path = os.path.join(save_path, "images", tmp_img_name)
            mask_path = os.path.join(save_path, "masks", tmp_mask_name)

            tifffile.imwrite(image_path, i)
            tifffile.imwrite(mask_path, m)

            idx += 1

if __name__ == "__main__":
    """ Loading original images and masks. """
    path = "/content/gdrive/MyDrive/Colab Notebooks/Data_Augmentation/test"                      #Respective folder(s) in the drive
    images, masks = load_data(path)
    print(f"Original Images: {len(images)} - Original Masks: {len(masks)}")

    """ Creating folders. """
    create_dir("/content/gdrive/MyDrive/Colab Notebooks/new_data_test/images")
    create_dir("/content/gdrive/MyDrive/Colab Notebooks/new_data_test/masks")

    """ Applying data augmentation. """
    augment_data(images, masks, "/content/gdrive/MyDrive/Colab Notebooks/new_data_test", augment=True)

    """ Loading augmented images and masks. """
    images, masks = load_data("/content/gdrive/MyDrive/Colab Notebooks/new_data_test/")
    print(f"Augmented Images: {len(images)} - Augmented Masks: {len(masks)}")

Original Images: 25 - Original Masks: 25


100%|██████████| 25/25 [00:20<00:00,  1.23it/s]

Augmented Images: 150 - Augmented Masks: 150


# Import necessary libraries

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Concatenate, Dropout
from tensorflow import keras
from tensorflow.keras.optimizers import Adam



# Data Preprocessing

Set the working directory

In [ ]:
os.chdir(r'/content/gdrive/MyDrive/Colab Notebooks/Modeling')          #Respective folder(s) in the drive

Now, all the images and masks (original and augmented) for training are stored in one folder named 'training', and all the images and masks (original and augmented) for testing are stored in another folder named 'testing'. These training and testing folders are kept inside the 'Modeling' folder.

In [ ]:
train_x = sorted(glob.glob('training/*_i.tif'))
train_y = sorted(glob.glob('training/*_l.tif'))
test_x = sorted(glob.glob('testing/*_i.tif'))
test_y = sorted(glob.glob('testing/*_l.tif'))

print(len(train_x), len(test_x))

600 150


Converting training image to numpy array

In [ ]:
train_xx = np.zeros((600, 256, 256, 3))
train_yy = np.zeros((600, 256, 256, 1))
for i, (img, mask) in enumerate(zip(train_x, train_y)):
  # if i == 1:
  #   break

  img = Image.open(img)
  np_img = np.array(img)
  train_xx[i] = np_img

  mask = Image.open(mask)
  np_mask = np.array(mask).reshape(256, 256, 1)
  train_yy[i] = np_mask

Converting testing image to numpy array

In [ ]:
test_xx = np.zeros((150, 256, 256, 3))
test_yy = np.zeros((150, 256, 256, 1))
for i, (img, mask) in enumerate(zip(test_x, test_y)):

  img = Image.open(img)
  np_img = np.array(img)
  test_xx[i] = np_img

  mask = Image.open(mask)
  np_mask = np.array(mask).reshape(256, 256, 1)
  test_yy[i] = np_mask

Let's plot a sample input RGB image and output image with classes

In [ ]:
plt.imshow(train_xx[500, :, :, :].astype('uint8'))
plt.show()
plt.imshow(train_yy[500, :, :, 0].astype('uint8'))
plt.show()

# Save to numpy format

In [ ]:
train_xx.shape

(600, 256, 256, 3)

In [ ]:
train_yy.shape

(600, 256, 256, 1)

In [ ]:
np.save('train_xx.npy', train_xx)
np.save('train_yy.npy', train_yy)
np.save("test_xx.npy", test_xx)
np.save("test_yy.npy", test_yy)

# Data for land use land cover mapping

Input data are RGB satellite images. And output are images of land cover type. There are 6 land cover types as below,

- Pixel value 1: Road
- Pixel value 2: Building
- Pixel value 3: Needle leaf
- Pixel value 4: Broad leaf
- Pixel value 5: Barren land
- Pixel value 6: Unlabelled


Data is already randomized and split in to training / test sets. So we can go ahead and use them as it is.

In [ ]:
x_train = np.load('train_xx.npy').astype('float32')
y_train= np.load('train_yy.npy').astype('float32')
x_test = np.load('test_xx.npy').astype('float32')
y_test = np.load('test_yy.npy').astype('float32')

print("x_train shape", x_train.shape)
print("y_train shape", y_train.shape)
print("y_test shape", x_test.shape)
print("y_test shape", y_test.shape)

x_train shape (600, 256, 256, 3)
y_train shape (600, 256, 256, 1)
y_test shape (150, 256, 256, 3)
y_test shape (150, 256, 256, 1)


Let's plot a sample input RGB image and output image with land cover

In [ ]:
plt.imshow(x_test[100,:,:,:].astype('uint8'))
plt.show()

plt.imshow(y_test[100,:,:,0].astype('uint8'))
plt.show()

Since land cover data include classes, let's perform one-hot encoding first.

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_1hot = to_categorical(y_train)
y_test_1hot = to_categorical(y_test)

y_train_1hot.shape

(600, 256, 256, 7)

# Model development (Unet)

In [ ]:

x_in = Input(shape=(256, 256, 3))

'''Encoder'''
x_temp = Conv2D(32, (3, 3), activation='relu', padding='same')(x_in)
x_temp = Dropout(0.25)(x_temp)
x_skip1 = Conv2D(32, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = MaxPooling2D((2,2))(x_skip1)
x_temp = Conv2D(32, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = Dropout(0.25)(x_temp)
x_skip2 = Conv2D(32, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = MaxPooling2D((2,2))(x_skip2)
x_temp = Conv2D(64, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = Dropout(0.25)(x_temp)
x_skip3 = Conv2D(64, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = MaxPooling2D((2,2))(x_skip3)
x_temp = Conv2D(64, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = Dropout(0.5)(x_temp)
x_temp = Conv2D(64, (3, 3), activation='relu', padding='same')(x_temp)

'''Decoder'''
x_temp = Conv2DTranspose(64, (3, 3), activation='relu',  padding='same')(x_temp)
x_temp = Dropout(0.5)(x_temp)
x_temp = Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu',  padding='same')(x_temp)
x_temp = Concatenate()([x_temp, x_skip3])
x_temp = Conv2DTranspose(64, (3, 3), activation='relu',  padding='same')(x_temp)
x_temp = Dropout(0.5)(x_temp)
x_temp = Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu',  padding='same')(x_temp)
x_temp = Concatenate()([x_temp, x_skip2])
x_temp = Conv2DTranspose(32, (3, 3), activation='relu',  padding='same')(x_temp)
x_temp = Dropout(0.5)(x_temp)
x_temp = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu',  padding='same')(x_temp)
x_temp = Concatenate()([x_temp, x_skip1])
x_temp = Conv2DTranspose(32, (3, 3), activation='relu',  padding='same')(x_temp)
x_temp = Dropout(0.5)(x_temp)
x_temp = Conv2DTranspose(32, (3, 3), activation='relu',  padding='same')(x_temp)

'''Use 1 by 1 Convolution to get desired output bands'''
x_temp = Conv2D(32, (1, 1), activation='relu', padding='same')(x_temp)
x_temp = Conv2D(32, (1, 1), activation='relu', padding='same')(x_temp)
x_out = Conv2D(7, (1, 1), activation='softmax', padding='same')(x_temp)

model = Model(inputs=x_in, outputs=x_out)

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 256, 256, 32  0           ['conv2d[0][0]']                 
                                )                                                             

In [ ]:
history = model.fit(x_train, y_train_1hot, validation_data=(x_test, y_test_1hot), epochs=10, batch_size=50, verbose=1)

Epoch 1/80
12/12 [==============================] - 13s 1s/step - loss: 0.5190 - accuracy: 0.8210 - val_loss: 1.5131 - val_accuracy: 0.5489
Epoch 2/80
12/12 [==============================] - 12s 1s/step - loss: 0.5682 - accuracy: 0.7918 - val_loss: 1.1316 - val_accuracy: 0.6292
Epoch 3/80
12/12 [==============================] - 12s 997ms/step - loss: 0.5166 - accuracy: 0.8203 - val_loss: 0.9474 - val_accuracy: 0.6904
Epoch 4/80
12/12 [==============================] - 12s 970ms/step - loss: 0.4905 - accuracy: 0.8280 - val_loss: 0.9058 - val_accuracy: 0.7168
Epoch 5/80
12/12 [==============================] - 12s 1s/step - loss: 0.4770 - accuracy: 0.8322 - val_loss: 0.7971 - val_accuracy: 0.7398
Epoch 6/80
12/12 [==============================] - 12s 990ms/step - loss: 0.4663 - accuracy: 0.8349 - val_loss: 0.8585 - val_accuracy: 0.7234
Epoch 7/80
12/12 [==============================] - 12s 1s/step - loss: 0.4618 - accuracy: 0.8355 - val_loss: 0.8295 - val_accuracy: 0.7200
Epoch 8/80


Plot the loss

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.plot(history.history['accuracy'])
ax1.plot(history.history['val_accuracy'])
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.set_title('Accuracy over epoch')
ax1.legend(['Train', 'Test'], loc='upper left')

ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title('Loss over epoch')
ax2.legend(['Train', 'Test'], loc="upper right")

# Save the figure
plt.savefig('/content/gdrive/MyDrive/Colab Notebooks/Modeling/figure.png')             #Respective folder(s) in the drive

#Prediction

In [ ]:
'''Prediction over the test dataset'''
pred_test = model.predict(x_test)

pred_test = np.argmax(pred_test, axis=-1)
print(pred_test.shape)

5/5 [==============================] - 1s 142ms/step
(150, 256, 256)


Save the predicted images in the directory

In [ ]:
from PIL import Image
import numpy as np
import os

# Create a directory to store the predicted images
save_dir = '/content/gdrive/MyDrive/Colab Notebooks/predicted_images_withAugmentation'
os.makedirs(save_dir, exist_ok=True)

# Save each predicted image to the directory
for i in range(pred_test.shape[0]):
    image = pred_test[i, :, :]
    image = np.uint8(image * 255)  # Convert to uint8
    image = np.squeeze(image)  # Remove the single-channel dimension if present
    image_path = os.path.join(save_dir, f'predicted_image_{i}.png')
    Image.fromarray(image).save(image_path)

print("Predicted images saved successfully.")

Predicted images saved successfully.


let's compare sample predicted and actual land cover image with input RGB image

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(pred_test[48, :, :])
plt.title('Predicted Mask (Augmented)')
plt.show()
#plt.savefig('/content/gdrive/MyDrive/Colab Notebooks/Modeling/figurePA.png')
plt.imshow(y_test[48, :, :, 0])
plt.title('Original Mask')
plt.show()
#plt.savefig('/content/gdrive/MyDrive/Colab Notebooks/Modeling/figureOM.png')
plt.imshow(x_test[48,:,:,:].astype('uint8'))
plt.title('Original Tile')
plt.show()
#plt.savefig('/content/gdrive/MyDrive/Colab Notebooks/Modeling/figureOT.png')